In [1]:
%load_ext autoreload
%autoreload 2
%load_ext jupyter_black

In [13]:
import sys
from pathlib import Path
from pprint import pprint
from typing import Any, Dict, Optional
from datetime import datetime
import re
import boto3

sys.path.append(str(Path.cwd().parent))
from paper_bridge.indexer.configs import Config
from paper_bridge.indexer.src import (
    EnvVars,
    NeptuneClient,
    OpenSearchClient,
    get_ssm_param_value,
)

In [3]:
config = Config.load()
profile_name = EnvVars.AWS_PROFILE_NAME.value
boto3_session = boto3.Session(
    region_name=config.resources.default_region_name, profile_name=profile_name
)

neptune_endpoint = get_ssm_param_value(
    boto3_session,
    f"/{config.resources.project_name}-{config.resources.stage}/neptune/endpoint",
)
open_search_endpoint = get_ssm_param_value(
    boto3_session,
    f"/{config.resources.project_name}-{config.resources.stage}/opensearch/endpoint",
)

print(
    f"Neptune endpoint: {neptune_endpoint}\nOpenSearch endpoint: {open_search_endpoint}"
)

neptune_client = NeptuneClient(neptune_endpoint)
open_search_clients = [
    OpenSearchClient(
        host=open_search_endpoint.replace("http://", "").replace("https://", ""),
        port=443,
        index=index,
        region_name=config.resources.default_region_name,
        boto3_session=boto3_session,
    )
    for index in ["chunk", "statement"]
]

Neptune endpoint: paper-bridge-dev.cluster-cyq3catzzgsc.us-west-2.neptune.amazonaws.com
OpenSearch endpoint: https://e8ry2xvp7na4j1lz2e82.us-west-2.aoss.amazonaws.com


### Paper Count by Date

In [34]:
query = """
g.V().hasLabel('__Source__')
  .group()
    .by('base_date')
    .by(values('paper_id').dedup().count())
  .unfold()
  .order().by(keys, asc)
  .fold()
"""
result = neptune_client.client.submit(query).all().result()

pprint(result[0])

[{'2025-02-10': 3},
 {'2025-02-11': 3},
 {'2025-02-12': 3},
 {'2025-02-13': 3},
 {'2025-02-14': 3},
 {'2025-02-17': 3},
 {'2025-02-18': 3},
 {'2025-02-19': 3},
 {'2025-02-20': 2},
 {'2025-02-21': 3},
 {'2025-02-24': 3},
 {'2025-02-25': 3},
 {'2025-02-26': 3},
 {'2025-02-27': 3},
 {'2025-02-28': 3},
 {'2025-03-03': 3},
 {'2025-03-04': 3},
 {'2025-03-05': 3},
 {'2025-03-06': 3},
 {'2025-03-07': 3},
 {'2025-03-09': 2}]


In [39]:
def get_paper_id_count_by_date(client: OpenSearchClient) -> Dict[str, Any]:
    query = {
        "size": 10000,
        "_source": [
            "metadata.source.metadata.paper_id",
            "metadata.source.metadata.base_date",
        ],
    }

    result = client.client.search(index=client.index, body=query)

    paper_ids_by_date = {}
    for hit in result.get("hits", {}).get("hits", []):
        source = hit.get("_source", {})
        metadata = source.get("metadata", {}).get("source", {}).get("metadata", {})

        date = metadata.get("base_date")
        paper_id = metadata.get("paper_id")

        if date and paper_id:
            if date not in paper_ids_by_date:
                paper_ids_by_date[date] = set()
            paper_ids_by_date[date].add(paper_id)

    date_counts = {
        date: len(paper_ids) for date, paper_ids in paper_ids_by_date.items()
    }

    sorted_date_counts = [{k: date_counts[k]} for k in sorted(date_counts.keys())]

    return sorted_date_counts

In [40]:
pprint(get_paper_id_count_by_date(open_search_clients[0]))
pprint(get_paper_id_count_by_date(open_search_clients[1]))

[{'2025-02-10': 3},
 {'2025-02-11': 3},
 {'2025-02-12': 3},
 {'2025-02-13': 3},
 {'2025-02-14': 3},
 {'2025-02-17': 3},
 {'2025-02-18': 3},
 {'2025-02-19': 3},
 {'2025-02-20': 2},
 {'2025-02-21': 3},
 {'2025-02-24': 3},
 {'2025-02-25': 3},
 {'2025-02-26': 3},
 {'2025-02-27': 3},
 {'2025-02-28': 3},
 {'2025-03-03': 3},
 {'2025-03-04': 3},
 {'2025-03-05': 3},
 {'2025-03-06': 3},
 {'2025-03-07': 3},
 {'2025-03-09': 2}]
[{'2025-02-10': 3},
 {'2025-02-11': 3},
 {'2025-02-12': 3},
 {'2025-02-13': 3},
 {'2025-02-14': 3},
 {'2025-02-17': 3},
 {'2025-02-18': 3},
 {'2025-02-19': 3},
 {'2025-02-20': 2},
 {'2025-02-21': 3},
 {'2025-02-24': 3},
 {'2025-02-25': 3},
 {'2025-02-26': 3},
 {'2025-02-27': 3},
 {'2025-02-28': 3},
 {'2025-03-03': 3},
 {'2025-03-04': 3},
 {'2025-03-05': 3},
 {'2025-03-06': 3},
 {'2025-03-07': 3},
 {'2025-03-09': 2}]


In [ ]:
# Query to show all edge labels in the graph
edge_labels_query = """
g.E().label().dedup().fold()
"""

edge_labels_result = neptune_client.client.submit(edge_labels_query).all().result()
print("All edge labels in the graph:")
print(edge_labels_result)

# Query to show incoming and outgoing edge labels for Topic nodes
topic_edge_labels_query = """
g.V().hasLabel('__Topic__').limit(1)
.project('node_id', 'incoming_edges', 'outgoing_edges')
.by(id())
.by(__.inE().label().dedup().fold())
.by(__.outE().label().dedup().fold())
"""

topic_edge_labels_result = (
    neptune_client.client.submit(topic_edge_labels_query).all().result()
)
print("\nEdge labels for Topic nodes:")
print(
    f"Incoming edges: {topic_edge_labels_result[0]['incoming_edges'] if topic_edge_labels_result else []}"
)
print(
    f"Outgoing edges: {topic_edge_labels_result[0]['outgoing_edges'] if topic_edge_labels_result else []}"
)

In [ ]:
base_date = "2025-03-01"
query = f"""
g.V().hasLabel('__Source__').has('base_date', '{base_date}')
            .values('paper_id').dedup().fold()
"""
a = neptune_client.client.submit(statements_query).all().result()

In [ ]:
a

In [ ]:
statements_query = """
            g.V().has('__Source__', 'paper_id', '{paper_id}')
            .in('__FROM_SOURCE__').hasLabel('__Topic__')
            .in('__FROM_TOPIC__').hasLabel('__Statement__').dedup().fold()
            """

statements_result = neptune_client.client.submit(statements_query).all().result()

In [ ]:
identification_result

In [ ]:
orphaned_facts_query = """
g.V().has('__Source__', 'paper_id', '{paper_id}')
.in('__FROM_SOURCE__').hasLabel('__Topic__')
.in('__FROM_TOPIC__').hasLabel('__Statement__').dedup()
.in('__SUPPORTS__').hasLabel('__Fact__')
.filter(__.out('__SUPPORTS__').not(
    __.until(__.not(__.in('__FROM_TOPIC__').in('__FROM_SOURCE__').has('__Source__', 'paper_id', '{paper_id}')))
    .repeat(__.identity())
    .count().is(0)
))
.dedup().fold()
"""

orphaned_facts_result = (
    neptune_client.client.submit(orphaned_facts_query).all().result()
)

In [ ]:
orphaned_entities_query = """
g.V().has('__Source__', 'paper_id', '{paper_id}')
.in('__FROM_SOURCE__').hasLabel('__Topic__')
.in('__FROM_TOPIC__').hasLabel('__Statement__').dedup()
.in('__SUPPORTS__').hasLabel('__Fact__')
// Only include facts where ALL supported statements are from this source
.filter(__.out('__SUPPORTS__').not(
    __.until(__.not(__.in('__FROM_TOPIC__').in('__FROM_SOURCE__').has('__Source__', 'paper_id', '{paper_id}')))
    .repeat(__.identity())
    .count().is(0)
))
// Get connected entities
.project('subject_entities', 'object_entities')
.by(__.out('__SUBJECT__').hasLabel('__Entity__')
    // Only include entities that would become orphaned
    .filter(__.in('__SUBJECT__', '__OBJECT__')
            .not(__.hasLabel('__Fact__').not(
                // Filter to only include facts we're NOT deleting
                __.until(__.in('__SUPPORTS__').in('__FROM_TOPIC__').in('__FROM_SOURCE__').has('__Source__', 'paper_id', '{paper_id}'))
                .repeat(__.identity())
                .count().is(0)
            ))
            .count().is(0)
    ).fold())
.by(__.out('__OBJECT__').hasLabel('__Entity__')
    // Only include entities that would become orphaned
    .filter(__.in('__SUBJECT__', '__OBJECT__')
            .not(__.hasLabel('__Fact__').not(
                // Filter to only include facts we're NOT deleting
                __.until(__.in('__SUPPORTS__').in('__FROM_TOPIC__').in('__FROM_SOURCE__').has('__Source__', 'paper_id', '{paper_id}'))
                .repeat(__.identity())
                .count().is(0)
            ))
            .count().is(0)
    ).fold())
"""

orphaned_entities_result = (
    neptune_client.client.submit(orphaned_entities_query).all().result()
)

In [ ]:
statements_result

In [ ]:
# 특정 paper_id를 가진 노드 찾기
paper_id = "2502.18417"
query = f"g.V().has('paper_id', '{paper_id}').valueMap(true)"

print(f"Searching for nodes with paper_id = {paper_id}")
result = neptune_client.client.submit_async(query).result()

# 결과 확인
nodes = list(result)
if nodes:
    print(f"Found {len(nodes)} nodes with paper_id = {paper_id}:")
    for node in nodes:
        print(node)
else:
    print(f"No nodes found with paper_id = {paper_id}")

In [ ]:
# paper_id가 "2502.18417"인 노드 ID 찾기
paper_id = "2502.18417"
find_nodes_query = f"g.V().has('paper_id', '{paper_id}').id()"
paper_nodes_result = neptune_client.client.submit_async(find_nodes_query).result()
paper_nodes = list(paper_nodes_result)

if paper_nodes:
    print(f"Found {len(paper_nodes)} nodes with paper_id = {paper_id}:")

    # 각 노드에 대해 참조하는 노드들 찾기
    for node_id in paper_nodes:
        print(f"Node ID: {node_id}")

        try:
            # 이 노드를 참조하는 노드들 (incoming edges)
            # Fix the query to properly escape the node_id
            referencing_query = f'g.V("{node_id}").in().valueMap(true)'
            referencing_result = neptune_client.client.submit_async(
                referencing_query
            ).result()
            referencing_nodes = list(referencing_result)

            print(
                f"  Nodes referencing this node (incoming edges): {len(referencing_nodes)}"
            )
            for ref_node in referencing_nodes:
                print(f"  {ref_node}")
        except Exception as e:
            print(f"  Error querying references for node {node_id}: {str(e)}")
else:
    print(f"No nodes found with paper_id = {paper_id}")

In [ ]:
node_id

In [ ]:
"aws:69ef7317:c0ce" -> __Chunk__

In [ ]:
# 특정 ID를 가진 노드와 연결된 모든 노드 조회
# node_id = "aws:69ef7317:c0ce"
# node_id = "aws:69ef7317:c0ce:ca5702ea"
node_id = "aws:69ef7317:c0ce:ca5702ea"

# 이 노드를 참조하는 노드들 (incoming edges)
incoming_query = f"g.V('{node_id}').in().valueMap(true)"
incoming_result = neptune_client.client.submit_async(incoming_query).result()
incoming_nodes = list(incoming_result)

# 이 노드가 참조하는 노드들 (outgoing edges)
outgoing_query = f"g.V('{node_id}').out().valueMap(true)"
outgoing_result = neptune_client.client.submit_async(outgoing_query).result()
outgoing_nodes = list(outgoing_result)

print(f"Nodes referencing {node_id} (incoming edges): {len(incoming_nodes)}")
for node in incoming_nodes:
    print(node)

print(f"\nNodes referenced by {node_id} (outgoing edges): {len(outgoing_nodes)}")
for node in outgoing_nodes:
    print(node)

In [ ]:
incoming_nodes[0]

In [ ]:
incoming_nodes

In [ ]:
# aws:69ef7317:c0ce로 시작하는 모든 노드 조회
prefix = "aws:69ef7317:c0ce"
# Neptune에서는 startingWith 함수가 없음. 대신 문자열 리터럴로 처리
query = f"g.V().has('id', '{prefix}')"

print(f"Searching for nodes with ID starting with '{prefix}'")
try:
    result = neptune_client.client.submit_async(query).result()

    # 결과 확인
    nodes = list(result)
    if nodes:
        print(f"Found {len(nodes)} nodes with ID starting with '{prefix}':")
        for node in nodes:
            print(node)
    else:
        print(f"No nodes found with ID starting with '{prefix}'")
except Exception as e:
    print(f"Error querying nodes: {e}")

# 특정 ID를 직접 조회하는 방식으로 변경
print("\nChecking specific node ID:")
specific_id = "aws:69ef7317:c0ce:ca5702ea"
id_query = f"g.V('{specific_id}')"
try:
    id_result = neptune_client.client.submit_async(id_query).result()
    node_ids = list(id_result)
    if node_ids:
        print(f"Found node with ID '{specific_id}':")
        for node_id in node_ids:
            print(node_id)
    else:
        print(f"No node found with ID '{specific_id}'")
except Exception as e:
    print(f"Error querying node ID: {e}")

In [ ]:
# 모든 노드를 조회하는 쿼리 실행
query = "g.V().count()"
result = neptune_client.client.submit_async(query).result()
node_count = result.one()
print(f"Total number of nodes in the graph: {node_count}")

# 노드 샘플 10개 조회
# Fix the query syntax - Neptune doesn't support with_(WithOptions.tokens)
sample_query = (
    "g.V().has('__Source__').valueMap(true)"  # true parameter includes vertex labels
)
sample_result = neptune_client.client.submit_async(sample_query).result()
print("\nSample nodes:")
for node in sample_result:
    print(node)

In [ ]:
sample_result = neptune_client.client.submit_async(sample_query).result()
a = list(sample_result)

In [ ]:
a[0]

In [ ]:
# 모든 노드를 삭제하는 쿼리 실행
print("Deleting all nodes from the graph...")
delete_query = "g.V().drop()"
result = neptune_client.client.submit_async(delete_query).result()
print("All nodes have been deleted from the graph.")

# 삭제 후 노드 수 확인
count_query = "g.V().count()"
count_result = neptune_client.client.submit_async(count_query).result()
remaining_nodes = count_result.one()
print(f"Remaining nodes in the graph after deletion: {remaining_nodes}")

In [ ]:
count_query = "g.V().count()"
count_result = neptune_client.client.submit_async(count_query).result()
remaining_nodes = count_result.one()
print(f"Remaining nodes in the graph after deletion: {remaining_nodes}")

In [ ]:
import boto3
from gremlin_python.structure.graph import Graph
from gremlin_python.process.graph_traversal import __
from gremlin_python.process.strategies import *
from gremlin_python.driver.driver_remote_connection import DriverRemoteConnection
from gremlin_python.driver.aiohttp.transport import AiohttpTransport
from gremlin_python.process.traversal import *
from paper_bridge.indexer.configs import Config
from paper_bridge.indexer.src import EnvVars, get_ssm_param_value, logger

In [ ]:
config = Config.load()
profile_name = EnvVars.AWS_PROFILE_NAME.value
boto3_session = boto3.Session(
    region_name=config.resources.default_region_name, profile_name=profile_name
)

neptune_endpoint = get_ssm_param_value(
    boto3_session,
    f"/{config.resources.project_name}-{config.resources.stage}/neptune/endpoint",
)
open_search_endpoint = get_ssm_param_value(
    boto3_session,
    f"/{config.resources.project_name}-{config.resources.stage}/opensearch/endpoint",
)

print(
    f"Neptune endpoint: {neptune_endpoint}\nOpenSearch endpoint: {open_search_endpoint}"
)

In [ ]:
endpoint = f'wss://{neptune_endpoint}:8182/gremlin'

graph=Graph()

connection = DriverRemoteConnection(endpoint,'g',
                 transport_factory=lambda:AiohttpTransport(call_from_event_loop=True))

g = graph.traversal().withRemote(connection)

results = (g.V().hasLabel('airport')
                .sample(10)
                .order()
                .by('code')
                .local(__.values('code','city').fold())
                .toList())

# Print the results in a tabular form with a row index
for i,c in enumerate(results,1):
    print("%3d %4s %s" % (i,c[0],c[1]))

connection.close()

In [ ]:
gremlin_client = client.Client(
    f"wss://{neptune_endpoint}:8182/gremlin",
    "g",
    message_serializer=GraphSONSerializersV2d0(),
)

In [ ]:
results = gremlin_client.submit("g.V().limit(10)").all().result()
print(results)

In [ ]:
results = neptune_client.client.submit("g.V().valueMap(true)").all().result()
for vertex in results:
    print(vertex)

In [ ]:
query = """
g.V('aws:c8795c68:6054').project("vertex", "in_edges", "out_edges")
    .by(valueMap(true))
    .by(inE().valueMap(true).fold())
    .by(outE().valueMap(true).fold())
"""

results = gremlin_client.submit(query).all().result()
for r in results:
    print("\nVertex:", r["vertex"])
    print("Incoming edges:", r["in_edges"])
    print("Outgoing edges:", r["out_edges"])

In [ ]:
query = """
g.V('223db800-9dd8-4f1b-85af-a666149fe35c0').project("vertex", "in_edges", "out_edges")
    .by(valueMap(true))
    .by(inE().valueMap(true).fold())
    .by(outE().valueMap(true).fold())
"""

results = gremlin_client.submit(query).all().result()
for r in results:
    print("\nVertex:", r["vertex"])
    print("Incoming edges:", r["in_edges"])
    print("Outgoing edges:", r["out_edges"])

In [ ]:
try:
    node_id = "57d3d114-22a3-489f-bfac-f2ec37b722da0"

    # 모든 속성 조회 (숨겨진 속성 포함)
    query = f"""
    g.V('{node_id}')
        .project('properties', 'hidden_properties')
        .by(valueMap())
        .by(properties().key().dedup())
    """

    result = gremlin_client.submit(query).all().result()
    print("All properties:", result)

    # 메타데이터 속성도 조회
    meta_query = f"""
    g.V('{node_id}')
        .elementMap()
    """

    meta_result = gremlin_client.submit(meta_query).all().result()
    print("\nWith metadata:", meta_result)

except Exception as e:
    print(f"Error occurred: {str(e)}")

In [ ]:
query = "g.V().has('paper_id', '2502.14502').valueMap(true)"
result = gremlin_client.submit(query).all().result()
print("Node properties:", result)

In [ ]:


# Neptune 엔드포인트에 연결
connection = DriverRemoteConnection('wss://your-neptune-endpoint:8182/gremlin','g')
g = traversal().withRemote(connection)

# 데이터 조회 예시
# 모든 버텍스 조회
results = g.V().toList()

# 특정 레이블의 버텍스 조회
results = g.V().hasLabel('person').toList()

In [ ]:
try:
    # 모든 vertex와 edge 삭제
    query = "g.V().drop()"
    result = neptune_client.client.submit(query).all().result()
    print("All data has been deleted")

    # 확인을 위해 vertex 수 조회
    count_query = "g.V().count()"
    count = neptune_client.client.submit(count_query).all().result()
    print(f"Remaining vertices: {count}")

except Exception as e:
    print(f"Error occurred: {str(e)}")